<a href="https://colab.research.google.com/github/Saharsh11/Wine_Wizard/blob/master/Wine_Wizard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from os import path
from sklearn.cluster import KMeans

from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.metrics.pairwise import linear_kernel

import csv
from csv import writer

!pip install -q fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

print("importing done")

importing done


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Using of fuzzy wuzzy to compare words of wine searched in the data.

Adding .CSV file to the system and pointing towards its header

In [0]:
wine = pd.read_csv('WineData.csv',index_col=0)
sample = wine.sample(n=501, random_state=21)
sample = sample.reset_index()
try_wine = wine

Creating global variables which will be used throughout the notebook. **Flavors** contains major flavor written in despcription, **drinkType** defining the alcohol content required by the user, **wineType** defining the type of wine as per data provided.

In [0]:
flavors = ['coffee','spice','fruit','floral','herb','cream','pepper',
            'chocolate','petrol','tabacco','black current','berry',
            'vanilla','diesel']

drinkType = ['light','medium','heavy']

wineType = {1: 'red',2: 'white',3: 'sparkling'}

Created s-words list so that we can skip these words while searching in description of wine.
the use of snowballStemmer is to 

In [0]:
s_words = ['wine', 'flavor', 'flavors', 'notes', 'offers', 
                            'enjoy', 'drink', 'bottling','really', 'reveal',
                            'picked', 'partner', 'thanks', 'grown', 'gain',
                            'thanks', 'percentage', 'buy', 'mid', 'gain',
                            'bottle', 'amid', 'available','come', 'aspect',
                            'try', 'make']

stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]


Now taking user details from the user.

In [0]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
        


In [0]:
#Created Vectorizer for the recommendation system
tf = TfidfVectorizer(stop_words=s_words,tokenizer=tokenize, max_features = 501)
tf_matrix = tf.fit_transform(wine['description'].values.astype('U'))
tf_matrix

cosineSim = linear_kernel(tf_matrix, tf_matrix) 
cosine = pd.DataFrame(cosineSim)
#cosine.head(10) #printing first 10 rows of the matrix
indices = pd.Series(wine.index,index = wine['title']).drop_duplicates()


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['avail', 'bottl', 'note', 'offer', 'percentag', 'pick', 'realli', 'thank', 'tri'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
#For matching wine names with previously entered wine
#print(wine)
wine1=wine
#print(len(wine))
def get_choice_by_ingredient(k1,win):
  print('win:',len(win))
  a=list()
  b=list()
  c=list()
  #The choice with whom we are matching
  ch = win['description']
  ti = win['title']
  print('ch:',len(ch),'ti:',len(ti))
  #extracting the names of wine that are matched or are similar to given name.
  i = -1
  j=0
  for txt in ch:
    i+=1
    option = fuzz.token_set_ratio(k1,txt)
    if(option == 100):
      b.append(ch[i])
      a.append(ti[i])
      c.append(i)
    else:
      continue
  return win[win.description.isin(b)]

#For testing of the function
#temp= get_choice_by_ingredient('vanilla',wine)
#print('total entries: ',len(temp),'\n')
#print(try_wine)
#print(wine)
#print(temp)
#print('Done filtering!!')
#wine2

In [0]:
#Function to filter the dataset filtering in terms of ingerdients.
print('enteries in wine1: ',len(wine1)) #Displaying total entries in the dataset
def check_multi_ingredients(wine):
  flavors = ['coffee','spice','fruit','floral','herb','cream','pepper',
            'chocolate','petrol','tabacco','black current','berry',
            'vanilla','diesel']
  added_flavor = []
  t = True
  sample_wine = wine #copy of dataset
  while(t):
    for flvr in flavors:
     print('| ',flvr,' | ')
    flavor = input('Enter from following flavors: ')
    if flavor in flavors:
      added_flavor.append(flavor)
      flavors.remove(flavor)
      rslt_wine = get_choice_by_ingredient(flavor, sample_wine)
      print('total entries after filtering: ',len(rslt_wine),'\n')
    else:
      print('please enter proper flavor name from above.')
    if(input('want to add more flavor(y/n):').casefold()=='n'):
      t=False

  return rslt_wine

#result_wine = check_multi_ingredients(wine)
#print('total entries in new Dataset: ',len(result_wine),'\n')
#print('Done filtering!!')

enteries in wine1:  501


In [0]:
#wine = result_wine
#Defining clusters of dataset
def alcohol_per(val, wine):
  winelight = wine[wine['alcohol_percentage']<=13.0]
  winemedium = wine[(16.0 >= wine['alcohol_percentage']) & (wine['alcohol_percentage']>13.0)]
  wineheavy = wine[wine['alcohol_percentage'] > 16.0]

  if(val == '1'):
    return winelight
  elif(val == '3'):
    return wineheavy
  else:
    return winemedium
#wine = wine2

#Storing data
#data = [name, age, gender, weight, country, alcoholType]
 
#Append a list as new line to an old csv file
#append_list_as_row('PersonInfo.csv', data)

#wine = alcohol_per(val)
#print('total wine items selected: ',len(wine))

In [0]:
#single wine name as per points
def single_per_points(win):
  win = win.sort_values(by='alcohol_percentage',ascending=False)
  win = win.sort_values(by = 'points',ascending=False)
  w1 = win.head(1)
  return w1

#winePoints = single_per_points(wine)
#print('Wine by points: ')
#print(winePoints['title'])

#single wine name as per price
def single_per_price(win):
  win = win.sort_values(by='alcohol_percentage',ascending=False)
  win = win.sort_values(by = 'price',ascending=True)
  w1 = win.head(1)
  return w1

#winePrice = single_per_price(wine)
#print('Wine by price: ')
#print(winePrice['title'])

In [0]:
#recommending 10 wine names as per points
#w = winePoints['title'].tolist()
#title = w[0]
def get_recommendations(title, cosineSim=cosineSim):
  idx = indices[title]
  simScores = list(enumerate(cosineSim[idx]))
  simScores = sorted(simScores, key=lambda x: x[1], reverse=True)
  simScores = simScores[0:5] #included the 0-th index
  wineIndices = [i[0] for i in simScores]
  return sample['title'].iloc[wineIndices]

#results = get_recommendations(title)
#results = results.tolist()
#results1 = try_wine[try_wine['title'].isin(results)]
#print('Recommendations as per Points: \n')
#for result in results1:
 # print(result,'\n')

In [0]:
#recommending 10 wine names as per price
#w = winePrice['title'].tolist()
#title = w[0]
def get_recommendations(title, cosineSim=cosineSim):
  idx = indices[title]
  simScores = list(enumerate(cosineSim[idx]))
  simScores = sorted(simScores, key=lambda x: x[1], reverse=True)
  simScores = simScores[0:5] #included the 0-th index 
  wineIndices = [i[0] for i in simScores]
  return sample['title'].iloc[wineIndices]

#results = get_recommendations(title)
#results = results.tolist()
#print('Recoomendations as per Price: \n')
#for result in results:
 # print(result,'\n')

In [0]:

def get_ch(key):
  ch = sample['title']
  option = fuzz.token_set_ratio(key,ch)
  if(option<50):
    matches = (process.extract(key, ch, scorer = fuzz.ratio, limit = 5))
    print('Choose among these 5 wines:')
    
    print("\n".join(map(str, matches)))
    inp = input('\nEnter wine: ')
  else:
    inp = key
  return process.extractOne(inp, ch, scorer = fuzz.token_sort_ratio, score_cutoff = 50)[0]

In [0]:
#Taking input from the user
wine = try_wine
name = str(input("Name: "))
age = input("Age: ")
gender = input("M=0, F=1 Gender: ")
weight = input("Weight: ")
country = str(input("Country: "))

cho = input('Do you have any wine preference(y/n):')
if(cho is 'y'):
  title = get_ch(input('Enter the wine you have tried:'))
  results = get_recommendations(title)
  results = results.tolist()

  print('Recommendations as per Points: \n')
  for result in results:
    print(result,'\n')

  results = get_recommendations(title)
  results = results.tolist()
  print('Recoomendations as per Price: \n')
  for result in results:
    print(result,'\n')
else:
  result_wine = check_multi_ingredients(wine)
  print('total entries in new Dataset: ',len(result_wine),'\n')
  print('Done filtering!!')
  
  print('|  light  | 1 | ','\n', '|  medium  | 2 | ','\n', '|  heavy  | 3 |','\n')
  val = input('enter the alcohol consumption level(1/2/3): ')
  wine = alcohol_per(val, result_wine)
  print('total wine items selected: ',len(wine))

  winePoints = single_per_points(wine)
  print('Wine by points: ')
  print(winePoints['title'])

  winePrice = single_per_price(wine)
  print('Wine by price: ')
  print(winePrice['title'])

  results = get_recommendations(winePoints['title'])
  results = results.tolist()

  print('Recommendations as per Points:')
  for result in results:
    print(result)
  print('\n')

  results = get_recommendations(winePrice['title'])
  results = results.tolist()
  print('Recoomendations as per Price:')
  for result in results:
    print(result)
  print('\n')

#Append a list as new line to an old csv file
#data = [name, age, gender, weight, country, alcoholType,]
#append_list_as_row('PersonInfo.csv', data)

Name: tom
Age: 30
M=0, F=1 Gender: 0
Weight: 70
Country: India
Do you have any wine preference(y/n):n
|  coffee  | 
|  spice  | 
|  fruit  | 
|  floral  | 
|  herb  | 
|  cream  | 
|  pepper  | 
|  chocolate  | 
|  petrol  | 
|  tabacco  | 
|  black current  | 
|  berry  | 
|  vanilla  | 
|  diesel  | 
Enter from following flavors: fruit
win: 501
ch: 501 ti: 501
total entries after filtering:  169 

want to add more flavor(y/n):y
|  coffee  | 
|  spice  | 
|  floral  | 
|  herb  | 
|  cream  | 
|  pepper  | 
|  chocolate  | 
|  petrol  | 
|  tabacco  | 
|  black current  | 
|  berry  | 
|  vanilla  | 
|  diesel  | 
Enter from following flavors: berry
win: 501
ch: 501 ti: 501
total entries after filtering:  66 

want to add more flavor(y/n):n
total entries in new Dataset:  66 

Done filtering!!
|  light  | 1 |  
 |  medium  | 2 |  
 |  heavy  | 3 | 

enter the alcohol consumption level(1/2/3): 2
total wine items selected:  47
Wine by points: 
index
290    Chateau Lafon-Rochet 2011  Sain